In [1]:
import pymongo
import pandas as pd
import time
import datetime
import xlwings as xw
import gc
# 連接MongoClient、Excel表，以及抓取CMoney的更新函數
client = pymongo.MongoClient()
db = client['Fields']

In [5]:
# 更新DB
def update_mongo(df: pd.DataFrame, dbname: str) -> None:
    for i, row in df.iterrows():
        db[dbname].update_one(filter={'日期': row['日期']}, update={'$set': row.to_dict()}, upsert=True)
path = r'C:\Users\ChastLai\Documents\Data\類股.xlsx'
#path = r'C:\Users\ChastLai\Documents\Data\選擇權oi.xlsx'
#wb1 = xw.Book(path)
#func_all = wb1.macro("CM全部更新")
#func_all()
#wb1.save()
#wb1.close()
exl = pd.read_excel(path, sheet_name=None, header=4)
for k, v in exl.items():
    print('D processing.. ', k)
    D = v.drop('股票名稱', axis=1).set_index('股票代號')
    #D = v.drop('名稱', axis=1).set_index('代號')
    D.columns = [s[:8] for s in D.columns]
    D.columns = pd.to_datetime(D.columns)
    D = D.T.dropna(how='all', axis=0)
    D.columns = D.columns.astype(str)
    D['日期'] = D.index
    update_mongo(D, k)


D processing..  開盤價_指數
D processing..  最高價_指數
D processing..  最低價_指數
D processing..  收盤價_指數
D processing..  成交金額(千)_指數
D processing..  總市值(億)_指數
D processing..  本益比_指數
D processing..  股價淨值比_指數
D processing..  股本(百萬)_指數


In [3]:
D

,20060901,20060904,20060905,20060906,20060907,20060908,20060911,20060912,20060913,20060914,...,20140930,20140929,20140926,20140925,20140924,20140923,20140922,20140919,20140918,20140917
股票代號,,,,,,,,,,,,,,,,,,,,,
TWB11,69.59,69.89,70.56,70.01,69.85,70.49,70.71,69.92,69.86,69.83,...,149.32,151.38,150.72,154.25,150.66,151.00,153.07,154.36,155.42,154.37
TWB12,363.84,366.05,371.91,367.91,363.88,366.04,370.73,369.48,370.94,367.86,...,1278.39,1288.79,1280.93,1282.27,1268.18,1266.61,1288.57,1298.14,1298.27,1293.29
TWB13,144.08,143.23,144.00,143.58,143.17,143.42,143.76,143.35,143.59,143.99,...,217.39,219.49,217.48,219.37,218.05,219.84,222.34,223.71,223.92,223.19
TWB15,73.91,74.08,74.25,73.70,72.73,73.21,73.31,73.54,72.90,72.15,...,161.27,162.52,161.75,166.79,166.86,167.66,169.41,170.37,167.41,166.64
TWB19,146.39,147.32,148.03,146.51,146.13,146.24,143.25,143.39,141.65,141.27,...,180.32,180.36,180.57,183.08,182.59,181.67,183.48,182.17,182.79,182.01
TWB20,83.81,84.10,84.87,84.96,85.67,86.13,86.03,85.00,84.23,84.39,...,100.31,101.39,101.59,102.71,101.74,103.02,103.56,104.11,102.60,101.70
TWB21,88.23,88.34,88.84,87.00,86.67,87.84,88.15,88.12,88.97,88.93,...,353.43,353.60,352.43,356.67,354.47,356.25,358.78,360.92,358.29,359.61
TWB22,113.73,114.33,114.22,115.09,114.16,114.87,114.09,113.94,112.32,111.31,...,313.47,319.78,318.71,320.41,316.90,318.08,323.01,320.17,319.49,318.47
TWB23,289.21,294.24,297.05,293.56,290.00,291.44,294.40,293.75,293.26,293.29,...,348.64,351.26,348.32,356.25,354.33,357.49,361.25,363.76,360.89,358.36


In [9]:
# 季資料日期轉換
def change_Q(se: pd.Series):
    new = list()
    for s in se:
        if 'Q1' in s:
            s = s[:4] + '0515'
        elif 'Q2' in s:
            s = s[:4] + '0814'
        elif 'Q3' in s:
            s = s[:4] + '1114'
        elif 'Q4' in s:
            s = str(int(s[:4]) + 1) + '0331'
        new.append(s)
    return new

# 更新DB
def update_mongo(df: pd.DataFrame, dbname: str) -> None:
    for i, row in df.iterrows():
        db[dbname].update_one(filter={'日期': row['日期']}, update={'$set': row.to_dict()}, upsert=True)

path = r'C:\Users\ChastLai\Documents\Data\每日更新DB\季資料.xlsm'
wb1 = xw.Book(path)
func_all = wb1.macro("CM全部更新")
func_all()
wb1.save()
wb1.close()

exl = pd.read_excel(path, sheet_name=None, header=4)
for k, v in exl.items():
    print('Q processing.. ', k)
    Q = v.drop('股票名稱', axis=1).set_index('股票代號')
    Q.columns = change_Q(Q.columns)
    Q.columns = pd.to_datetime(Q.columns)
    Q = Q.T.dropna(how='all', axis=0)
    Q.columns = Q.columns.astype(str)
    Q['日期'] = Q.index
    update_mongo(Q, k)


path = r'C:\Users\ChastLai\Documents\Data\每日更新DB\年資料.xlsm'
wb1 = xw.Book(path)
func_all = wb1.macro("CM全部更新")
func_all()
wb1.save()
wb1.close()

exl = pd.read_excel(path, sheet_name=None, header=4)
for k, v in exl.items():
    print('Y processing.. ', k)
    Y = v.drop('股票名稱', axis=1).set_index('股票代號')
    Y.columns = [str(int(s[:4]) + 1) + '0331' for s in Y.columns]
    Y.columns = pd.to_datetime(Y.columns)
    Y = Y.T.dropna(how='all', axis=0)
    Y.columns = Y.columns.astype(str)
    Y['日期'] = Y.index
    update_mongo(Y, k)

path = r'C:\Users\ChastLai\Documents\Data\每日更新DB\日資料.xlsm'
wb1 = xw.Book(path)
func_all = wb1.macro("CM全部更新")
func_all()
wb1.save()
wb1.close()
exl = pd.read_excel(path, sheet_name=None, header=4)
for k, v in exl.items():
    print('D processing.. ', k)
    D = v.drop('股票名稱', axis=1).set_index('股票代號')
    D.columns = [s[:8] for s in D.columns]
    D.columns = pd.to_datetime(D.columns)
    D = D.T.dropna(how='all', axis=0)
    D.columns = D.columns.astype(str)
    D['日期'] = D.index
    update_mongo(D, k)

# %%
db1 = client['Index']
#wb1.close()
#wb2.close()
wb3 = xw.Book(r"C:\Users\ChastLai\Documents\Data\每日更新DB\CMoney指數.xlsm")
func_ = wb3.macro("CM全部更新")
func_()
wb3.save()
wb3.close()
TWA00 = pd.read_excel(r'C:\Users\ChastLai\Documents\Data\每日更新DB\Cmoney指數.xlsm', header=4, sheet_name='TWA00').iloc[:, 1:]
TWA00.rename({
    '開盤價': '還原開盤價', '最高價': '還原最高價', '最低價': '還原最低價', '收盤價': '還原收盤價',\
    '開盤價.1': '開盤價', '最高價.1': '最高價','最低價.1': '最低價', '收盤價.1': '收盤價' 
    }, axis=1, inplace=True)
VIX = pd.read_excel(r'C:\Users\ChastLai\Documents\Data\每日更新DB\Cmoney指數.xlsm', header=4, sheet_name='VIX').iloc[:, 1:]
for i in range(len(TWA00)):
    日期 = TWA00.iloc[i]['日期']
    db1['TWA00'].update_one({'日期': 日期}, update={'$set': TWA00.iloc[i].to_dict()}, upsert=True)
for i in range(len(VIX)):
    日期 = VIX.iloc[i]['日期']
    db1['VIX'].update_one({'日期': 日期}, update={'$set': VIX.iloc[i].to_dict()}, upsert=True)

In [2]:
# 季資料日期轉換
def change_Q(se: pd.Series):
    new = list()
    for s in se:
        if 'Q1' in s:
            s = s[:4] + '0515'
        elif 'Q2' in s:
            s = s[:4] + '0814'
        elif 'Q3' in s:
            s = s[:4] + '1114'
        elif 'Q4' in s:
            s = str(int(s[:4]) + 1) + '0331'
        new.append(s)
    return new

# 更新DB
def update_mongo(df: pd.DataFrame, dbname: str) -> None:
    for i, row in df.iterrows():
        db[dbname].update_one(filter={'日期': row['日期']}, update={'$set': row.to_dict()}, upsert=True)

path = r'C:\Users\ChastLai\Documents\Data\投資金額.xlsx'
# wb1 = xw.Book(path)
# func_all = wb1.macro("CM全部更新")
# func_all()
# wb1.save()
# wb1.close()

exl = pd.read_excel(path, sheet_name=None, header=4)
for k, v in exl.items():
    print('Q processing.. ', k)
    Q = v.drop('股票名稱', axis=1).set_index('股票代號')
    Q.columns = change_Q(Q.columns)
    Q.columns = pd.to_datetime(Q.columns)
    Q = Q.T.dropna(how='all', axis=0)
    Q.columns = Q.columns.astype(str)
    Q['日期'] = Q.index
    update_mongo(Q, k)

Q processing..  投資活動現金流量(千)_Q
